# Basic Info:
The Project Title is: The Epidemiology of COVID-19

Group member 1 
name: Austin Hickey
e-mail: U1041943@utah.edu
UID: U1041943

Group member 2
name: Spencer Sawas
e-mail: spencer.sawas@utah.edu
UID: U1065866

Group member 3
name: Marko Miholjcic
e-mail: u0984549@utah.edu
UID: u0984549

# Background and Motivation


Currently the world is facing a global pandemic with a new virus that has evolved and crossed over to humans. The virus known as COVID-19 affects the respiratory and cardiovascular systems by binding to the Ace-2 receptors. Ace-2 receptors are found throughout the cardiovascular and respiratory system. For this reason, the virus is extremely dangerous for susceptible populations. Currently, 30% of Americans suffer from cardiovascular diseases, 10% from diabetes, and 10% from asthma; these populations do not account for those who are immunocompromised or more susceptible for other reasons. 
The virus causes violent coughing, restricted breathing, inflammation, and cardiovascular hypertension (among other cardiovascular effects). Furthermore, it can cause pneumonia. Pneumonia can be fatal; especially if left untreated. 
Due to how transmissible the virus is, hospitals and healthcare workers across the world have been put to the test. The rapid spread of the virus has caused many hospitals to become overloaded with patients, with limited resources available. 

To try and curb the spread of the virus, countries across the world are temporarily shutting down and government officials have been recommended to social quarentine. Ripple effects have been detrimental to the economy. Millions of people have lost their jobs. People are dipping into their savings to pay rent, while others are unable to pay rent. Some economists have speculated an economic recession after the virus passes.  

# Project Objectives ***

Understand the severity of the global pandemic and predict the effects to come in the near future for states in the United States and countries around the world. Using predictive modeling techniques such as logistic regression, we will identify how long until the number of cases begins to plateau, how many cases there will be when it begins to plateau, and a prediction of the number of deaths a location will experience. 

We will use the predictive models to create plots to visualize the severity of the virus in the locations analyzed. Further, we will explore which states will be most heavily impacted. A geospatial map will be created to plot the spread of COVID-19 in the United States and identify the hospital beds per 1,000 for each state.

The benefits? 
What would you like to learn and accomplish?

These variables will provide an oppurtunity to perform a clustering analysis to determine if they are any realtionships between variables and the cases in a state. 

# Data


We will be using several sources of data in order to accurately represent the COVID-19 outbreaks and information relevant to analyzing contributing factors to the outbreak.

For the COVID-19 data we will use multiple APIs of a github repository to collect the number of cases, recoveries, and deaths over time for a number of countries and all of the US states. 
This data is a collection of data put together in csv format by John Hopkins Center for Systems Science and Engineering. Data extracted will be between the dates of January 22, 2020 and April 1, 2020. The github webpage is:
https://github.com/CSSEGISandData/COVID-19.

Below a single url is retrieved and placed into a dataframe in order to display the data that is imported from the API. Each day, the github repository updates the cumulative number of cases, recoveries, and deaths. The data also provides the province/state and country/region where the cases are occuring. 

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv'
response = requests.get(url)
response

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(url)
df

For our regression model to predict the number of deaths from COVID-19 that a particular country will experience we will be extracting data from the WHO Healthcare index pdf. A countries healthcare index is a good indicator of how adept their healthcare infrastructure is, and how healthy a country is overall. The WHO healthcare index takes into account myriad of variables to rank every nations healthcare system.
https://www.who.int/healthinfo/paper30.pdf

In [ ]:
import pdfplumber
import pandas as pd
with pdfplumber.open(r"C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\pdf_resources\world_health_index.pdf") as pdf:
    pages = pdf.pages[17]
    table = pages.extract_table()
    pages2 = pdf.pages[18]
    table2 = pages2.extract_table()
    pages3 = pdf.pages[19]
    table3 = pages3.extract_table()
    pages4 = pdf.pages[20]
    table4 = pages4.extract_table()
    
df = pd.DataFrame(table[1:], columns=table[1])
df.head()
df=df[['Rank','Member State', 'Index']]

data = pd.DataFrame(table2[1:], columns=table2[1])
data=data[['55','Albania', '0.774']]
data=data.rename(columns={'55':'Rank','Albania':'Member State', '0.774':'Index'})                
df=df.append(data,ignore_index=True)

data = pd.DataFrame(table3[1:], columns=table3[1])
data=data[['117','Uzbekistan', '0.599']]
data=data.rename(columns={'117':'Rank','Uzbekistan':'Member State', '0.599':'Index'})                 
df=df.append(data,ignore_index=True)

data = pd.DataFrame(table4[1:], columns=table4[1])
data=data[['178','Chad', '0.303']]
data=data.rename(columns={'178':'Rank','Chad':'Member State', '0.303':'Index'})                  
df=df.append(data,ignore_index=True)

for column in ["Rank", "Member State", 'Index']:
    df[column] = df[column].str.replace(" ", "")
    df[column] = df[column].replace('\n','', regex=True)
df=df.dropna()
df=df.drop([0])
df.set_index('Rank')
df['Rank']=df['Rank'].astype(int)
df['Index']=df['Index'].astype(float)
df['Member State']=df['Member State'].astype(str)
df

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)
    
df.to_csv(r'C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\CLEAN_WorldHealthIndex.csv', index=False)

The population and population densities are collected from an csv file downloaded from the following website: http://worldpopulationreview.com/. We gathered two csv files from the website. One csv file had data from the states while the other csv file had data for all of the countries in the world. 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/population_and_density_by_country.csv")
df

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/population_and_population_density_USA.csv")
df

The size of the existing healthcare systems for each state (number of hospital beds per 1,000 people). The following website was utilized: https://www.kff.org/other/state-indicator/beds-by-ownership/?currentTimeframe=0&selectedDistributions=statelocal-government&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

In [ ]:
import pandas as pd
import requests

In [ ]:
from bs4 import BeautifulSoup

url = 'https://www.kff.org/other/state-indicator/beds-by-ownership/?currentTimeframe=0&selectedDistributions=statelocal-government&print=true&sortModel=%7B%22colId%22:%22State%2FLocal%20Government%22,%22sort%22:%22asc%22%7D'
response = requests.get(url)
response

In [ ]:
file = open("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt", "w")
file.write(response.text)
file = open("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt", "r")
content = file.readlines()
content

Furthermore, we will use data gathered from each country in the United States. This excel file will provide us with a plethora of variables to exlpore, such as the number of physicians per county for each state. The link to the website where the excel document was downloaded from is: https://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\2020 County Health Rankings Data - v1.csv',skiprows=1)

# Ethical considerations


If this were published, and publicly visible; we would not want our project to induce mass hysteria for the states that have been identified to be most severely impacted. It would also be important that our project does not identify a state that is least impacted- and influence them to go against governmental restrictions that were put in place.

As we are working with an ongoing crisis we are using the data while also upholding the real concequences this data is having on hundreds of thousands of lives in America and across the world. Our use of this data is not meant to be insensitive but instead to try and highlight exactly how extreme this can become. 

# Data Processing

All data processing will be done within jupyter notebook running python software. The API, CSV, and data obtained from web scraping will all need to be loaded into a jupyter notebook and will converted from JSON, dictionary, and/or lists into a pandas data frame for processing and analysis.
The github repository had multiple formats for date and the abbreviations of locations where cases had occured. Therefore, most APIs had their format altered in order to create consistency between the dates and locations of the data.

In [ ]:
import requests
import pandas as pd

dates = ['01-22', '01-23', '01-24', '01-25', '01-26', '01-27', '01-28', '01-29', '01-30', '01-31']

df = pd.DataFrame()

for x in range(len(dates)):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + dates[x] + '-2020.csv'
    response = requests.get(str(url))
        
    df1 = pd.read_csv(url)
    df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[0:7] + "20")
    states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
          "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
          "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", 
          "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", 
          "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
          "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
          "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"]
    abb = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

    for x in range(51):
        df1["Province/State"] = df1["Province/State"].replace(states[x], abb[x])
    
        df1["Province/State"] = df1["Province/State"].replace("Chicago", "IL")    
    df = df.append(df1, ignore_index = True)

#df 

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/02-01-2020.csv'
response = requests.get(url)

df1 = pd.read_csv(url)
df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[0:1] + "/0" + x[2:6] + "20")
mask = df1['Last Update'] == "02/01/2020"
df1 = df1.loc[mask]

df1["Province/State"] = df1["Province/State"].fillna("none") 
df1['Province/State'] = df1['Province/State'].apply(lambda x : x[-2:])
df = df.append(df1, ignore_index = True)
#df

In [ ]:
dates2 = ['02-02', '02-03', '02-04', '02-05', '02-06', '02-07', '02-08', '02-09', '02-10', '02-11', '02-12',
        '02-13', '02-14', '02-15', '02-16', '02-17', '02-18', '02-19', '02-20', '02-21', '02-22', '02-23',
        '02-24', '02-25', '02-26', '02-27', '02-28', '02-29']

for x in range(len(dates2)):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + dates2[x] + '-2020.csv'
    response = requests.get(url)

    df1 = pd.read_csv(url)
    df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[6:10] + "/" + "2020")
    df1['Last Update'] = df1['Last Update'].apply(lambda x : x.replace("-", "/"))
    mask = df1['Last Update'] == df1['Last Update'][0]
    df1 = df1.loc[mask]

    df1["Province/State"] = df1["Province/State"].replace("Omaha, NE (From Diamond Princess)", "NE")
    df1["Province/State"] = df1["Province/State"].replace("Travis, CA (From Diamond Princess)", "CA")
    df1["Province/State"] = df1["Province/State"].replace("Lackland, TX (From Diamond Princess)", "TX")

    df1["Province/State"] = df1["Province/State"].fillna("none") 
    df1['Province/State'] = df1['Province/State'].apply(lambda x : x[-2:])

    df = df.append(df1, ignore_index = True)

#df

In [ ]:
dates3 = ['03-01', '03-02', '03-03', '03-04', '03-05', '03-06', '03-07', '03-08', '03-09']

for x in range(len(dates3)):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + dates3[x] + '-2020.csv'
    response = requests.get(url)

    df1 = pd.read_csv(url)
    df1 = df1[["Province/State", "Country/Region",  "Last Update", "Confirmed", "Deaths", "Recovered"]]
    df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[6:7] + "/0" + x[9] + "/" + "2020")
    df1['Last Update'] = df1['Last Update'].apply(lambda x : x.replace("-", "/"))
    mask = df1['Last Update'] == df1['Last Update'][0]
    df1 = df1.loc[mask]

    df1["Province/State"] = df1["Province/State"].replace("Omaha, NE (From Diamond Princess)", "NE")
    df1["Province/State"] = df1["Province/State"].replace("Travis, CA (From Diamond Princess)", "CA")
    df1["Province/State"] = df1["Province/State"].replace("Lackland, TX (From Diamond Princess)", "TX")
    df1["Province/State"] = df1["Province/State"].replace("Washington, D.C.", "DC")

    df1["Province/State"] = df1["Province/State"].fillna("none") 
    df1['Province/State'] = df1['Province/State'].apply(lambda x : x[-2:])

    df = df.append(df1, ignore_index = True)
    
#df

In [ ]:
dates4 = ['03-10', '03-11', '03-12', '03-13', '03-14', '03-15', '03-16', '03-17', 
        '03-18', '03-19', '03-20', '03-21']

for x in range(len(dates4)):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + dates4[x] + '-2020.csv'
    response = requests.get(url)

    df1 = pd.read_csv(url)
    df1 = df1[["Province/State", "Country/Region",  "Last Update", "Confirmed", "Deaths", "Recovered"]]
    df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[6:10] + "/" + "2020")
    df1['Last Update'] = df1['Last Update'].apply(lambda x : x.replace("-", "/"))
    mask = df1['Last Update'] == df1['Last Update'][0]
    df1 = df1.loc[mask]
    states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
          "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
          "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", 
          "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", 
          "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
          "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
          "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"]
    abb = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

    for x in range(51):
        df1["Province/State"] = df1["Province/State"].replace(states[x], abb[x])
    
    df = df.append(df1, ignore_index = True)
    
#df

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-22-2020.csv'
response = requests.get(url)

df1 = pd.read_csv(url)
df1 = df1.rename(columns={"Province_State": "Province/State", "Country_Region": "Country/Region", "Last_Update": "Last Update"})
df1 = df1[["Province/State", "Country/Region", "Last Update", "Confirmed", "Deaths", "Recovered"]]

df1['Last Update'] = df1['Last Update'].apply(lambda x :"0" + x[0:7] + "20")
mask = df1['Last Update'] == df1['Last Update'][0]
df1 = df1.loc[mask]

states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
          "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
          "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", 
          "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", 
          "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
          "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
          "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"]
abb = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

for x in range(51):
    df1["Province/State"] = df1["Province/State"].replace(states[x], abb[x])

df = df.append(df1, ignore_index = True)
#df

In [ ]:
dates5 = ['03-23', '03-24', '03-25', '03-26', '03-27', '03-28', 
        '03-29', '03-30', '03-31', '04-01']

for x in range(len(dates5)):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + dates5[x] + '-2020.csv'
    response = requests.get(url)
    
    df1 = pd.read_csv(url)
    df1 = df1.rename(columns={"Province_State": "Province/State", "Country_Region": "Country/Region", "Last_Update": "Last Update"})
    df1 = df1[["Province/State", "Country/Region", "Last Update", "Confirmed", "Deaths", "Recovered"]]

    df1['Last Update'] = df1['Last Update'].apply(lambda x : "0" + x[6:10] + "/2020")
    df1['Last Update'] = df1['Last Update'].apply(lambda x : x.replace("-", "/"))
    mask = df1['Last Update'] == df1['Last Update'][0]
    df1 = df1.loc[mask]
    states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
          "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
          "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", 
          "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", 
          "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
          "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
          "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"]
    abb = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

    for x in range(51):
        df1["Province/State"] = df1["Province/State"].replace(states[x], abb[x])

    df = df.append(df1, ignore_index = True)
    
df

In [ ]:
df.to_csv('/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/CDRdata.csv', index = False)

After saving the dataframe with all of the countries, the following lines of code, selected the desired countries and US states for analysis.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/CDRdata.csv")
#df

In [ ]:
df = pd.read_csv("/Users/spenc/OneDrive/Documents/University_of_Utah/2019-2020/DataScience/COMP5360Project/project_files/csv_files/CDRdata.csv")
#df

In [ ]:
df = pd.read_csv(r"C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\CDRdata.csv")
#df

In [ ]:
df.describe()

In [ ]:
df["Confirmed"] = df["Confirmed"].fillna(0)
df["Deaths"] = df["Deaths"].fillna(0)
df["Recovered"] = df["Recovered"].fillna(0)
#df

In [ ]:
df = df[["Country/Region", "Last Update", "Confirmed", "Deaths", "Recovered"]]
#df

In [ ]:
df["Country/Region"] = df["Country/Region"].replace("Mainland China", "China")
df["Country/Region"] = df["Country/Region"].replace("Korea, South", "South Korea")
df["Country/Region"] = df["Country/Region"].replace("UK", "United Kingdom")
#df

In [ ]:
countries = ['US', 'China', 'Italy', 'New Zealand', 'South Korea', 'United Kingdom', 'Iran', 'Australia', 'India', 'Peru']
df = df.loc[df['Country/Region'].isin(countries)]
#df

In [ ]:
df["Confirmed"] = df["Confirmed"].astype("int")
df["Deaths"] = df["Deaths"].astype("int")
df["Recovered"] = df["Recovered"].astype("int")
df

In [ ]:
df.to_csv('/Users/spenc/OneDrive/Documents/University_of_Utah/2019-2020/DataScience/COMP5360Project/project_files/csv_files/CLEANCDRData.csv', index = False)

In [ ]:
df.to_csv('/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/CLEANCDRData.csv', index = False)

In [ ]:
df.to_csv(r'C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\CLEANCDRData.csv', index = False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/CDRdata.csv")
#df

In [ ]:
df.describe()

In [ ]:
df["Confirmed"] = df["Confirmed"].fillna(0)
df["Deaths"] = df["Deaths"].fillna(0)
df["Recovered"] = df["Recovered"].fillna(0)
#df

In [ ]:
data = df[df["Country/Region"] == "US"]
#data

In [ ]:
abb = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

dataUS = data[data["Province/State"].isin(abb)]
dataUS

In [ ]:
dataUS.to_csv('/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/usaCLEANCDRData.csv', index = False)

Data had to be extracted from the pdf for each country.  

In [ ]:
import pdfplumber
import pandas as pd
with pdfplumber.open(r"C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\pdf_resources\world_health_index.pdf") as pdf:
    pages = pdf.pages[17]
    table = pages.extract_table()
    pages2 = pdf.pages[18]
    table2 = pages2.extract_table()
    pages3 = pdf.pages[19]
    table3 = pages3.extract_table()
    pages4 = pdf.pages[20]
    table4 = pages4.extract_table()
    
df = pd.DataFrame(table[1:], columns=table[1])
df.head()
df=df[['Rank','Member State', 'Index']]

data = pd.DataFrame(table2[1:], columns=table2[1])
data=data[['55','Albania', '0.774']]
data=data.rename(columns={'55':'Rank','Albania':'Member State', '0.774':'Index'})                
df=df.append(data,ignore_index=True)

data = pd.DataFrame(table3[1:], columns=table3[1])
data=data[['117','Uzbekistan', '0.599']]
data=data.rename(columns={'117':'Rank','Uzbekistan':'Member State', '0.599':'Index'})                 
df=df.append(data,ignore_index=True)

data = pd.DataFrame(table4[1:], columns=table4[1])
data=data[['178','Chad', '0.303']]
data=data.rename(columns={'178':'Rank','Chad':'Member State', '0.303':'Index'})                  
df=df.append(data,ignore_index=True)

for column in ["Rank", "Member State", 'Index']:
    df[column] = df[column].str.replace(" ", "")
    df[column] = df[column].replace('\n','', regex=True)
df=df.dropna()
df=df.drop([0])
df.set_index('Rank')
df['Rank']=df['Rank'].astype(int)
df['Index']=df['Index'].astype(float)
df['Member State']=df['Member State'].astype(str)
df

The countries included in the report were selected from the population data. The datatypes were changed to integers where appropriate. 

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/population_and_density_by_country.csv")
#df

In [ ]:
df = pd.read_csv("/Users/spenc/OneDrive/Documents/University_of_Utah/2019-2020/DataScience/COMP5360Project/project_files/csv_files/population_and_density_by_country.csv")
#df

In [ ]:
df = df[["name","Rank", "pop2019", "GrowthRate", "area", "Density"]]
#df

In [ ]:
countries = ['China','United States', 'Italy', 'New Zealand', 'South Korea', 'United Kingdom', 'Iran', 'Singapore', 'India', 'Peru']
df = df.loc[df['name'].isin(countries)]
df

In [ ]:
df['pop2019'] = df['pop2019'].astype('int')
df['area'] = df['area'].astype('int')
#df

In [ ]:
df.describe()

In [ ]:
df.to_csv(r'/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/CLEANpopulation_and_density_by_country.csv', index = False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/population_and_population_density_USA.csv")
#df

In [ ]:
df.describe()

In [ ]:
#People per square mile

In [ ]:
df2 = pd.read_csv('/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files//csv_files/CLEANpopulation_and_population_density_USA.csv')

In [ ]:
df2 = pd.read_csv('/Users/spenc/OneDrive/Documents/University_of_Utah/2019-2020/DataScience/COMP5360Project/project_files/csv_files/CLEANpopulation_and_population_density_USA.csv')

The data in the file containing the size of the existing healthcare systems for each state (number of hospital beds per 1,000 people) was extracted. The website had a figure that contained all of the hospital bed data for each state. Beautiful soup and string concatenation were utilized in order to scrap the information from the figure on the website. 

In [ ]:
import pandas as pd
import requests

In [ ]:
from bs4 import BeautifulSoup

url = 'https://www.kff.org/other/state-indicator/beds-by-ownership/?currentTimeframe=0&selectedDistributions=statelocal-government&print=true&sortModel=%7B%22colId%22:%22State%2FLocal%20Government%22,%22sort%22:%22asc%22%7D'
response = requests.get(url)
response

In [ ]:
file = open("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt", "w")
file.write(response.text)
file = open("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt", "r")
content = file.readlines()
#content

In [ ]:
BigSoup = BeautifulSoup()

soup = BeautifulSoup(open("/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt"), "html.parser")
BigSoup.append(soup)

print(BigSoup.prettify())

In [ ]:
littlesoup = BigSoup.find(id = "content")
ls = str(littlesoup)
print(ls)

indexS = ls.find('["Alabama"')
indexE = ls.find(']],["2017"')

data_s = ls[indexS:indexE]


In [ ]:
#print(data_s)

In [ ]:
data_s = "[" + data_s + "]"
#data_s

In [ ]:
import ast
data = ast.literal_eval(data_s)

In [ ]:
#data

In [ ]:
columns = ["Location","State\Local Government","Non-Profit","For-Profit", "Total"]

df = pd.DataFrame(data, columns = columns)
#df

In [ ]:
df = df[["Location", "Total"]]
df

In [ ]:
df.dtypes

In [ ]:
df.to_csv(r'/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/pdf_resources/HospitalBed.txt', index = False)

The Country Data had to be processed. 27 variables were chosen for the County Data and were placed in a dataframe. The empty entires were filled with 0 values. 

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\2020 County Health Rankings Data - v1.csv',skiprows=1)
df=df[['State','County','Deaths','Years of Potential Life Lost Rate','Average Number of Physically Unhealthy Days','Average Number of Mentally Unhealthy Days','% Smokers','% Adults with Obesity','Food Environment Index','% Physically Inactive','% With Access to Exercise Opportunities','Chlamydia Rate','% Uninsured','# Primary Care Physicians','Primary Care Physicians Rate','Preventable Hospitalization Rate','% Vaccinated','High School Graduation Rate','% Some College','% Unemployed','% Children in Poverty','Income Ratio','% Single-Parent Households','Social Association Rate','Average Daily PM2.5','% Severe Housing Problems','% Long Commute - Drives Alone']]
df.set_index('State')
df['County'].fillna(0,inplace=True)

statelvl=df.loc[df['County']==0]
statelvl=statelvl.drop(['County'],axis=1)

#Nan pm2.5 values filled in with this source: http://berkeleyearth.lbl.gov/air-quality/local/United_States_of_America/Hawaii
statelvl.loc[statelvl['State']=='Alaska','Average Daily PM2.5']=12
statelvl.loc[statelvl['State']=='Hawaii','Average Daily PM2.5']=3.9 
#statelvl.dtypes
statelvl.to_csv(r'C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\Nation_HealthSurvey_Variables.csv', index=False)

# Exploratory Analysis

Plotted a time series for the selected states. 

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
data = pd.read_csv(r"/Users/markomiholjcic/Documents/GitHub/COMP5360Project/project_files/csv_files/usaCLEANCDRData.csv")
#data

In [ ]:
data = pd.read_csv(r"C:\Users\austi\Documents\GitHub\COMP5360Project\project_files\csv_files\usaCLEANCDRData.csv")

In [ ]:
date1 = '01/22/2020'
date2 = '04/01/2020'
mydates = pd.date_range(date1, date2).tolist()
#mydates

In [ ]:
dates = pd.DataFrame(mydates)
dates = dates.rename(columns = {0:"Date1"})
#dates = dates.set_index("Date")

dates['Date'] = dates['Date1'].dt.strftime('%m/%d/%Y')
dates = dates.set_index("Date")
#dates

In [ ]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

df = pd.DataFrame()

for x in range(len(states)):
    dataU = data[data["Province/State"] == states[x]]  
    groupDate = dataU.groupby("Last Update")
    df1 = groupDate.count()

    df1 = list(df1.index.values)
    df1.sort(key = lambda date: datetime.strptime(date, '%m/%d/%Y'))

    dfDate = pd.DataFrame(df1)
    
    dataC = data[data["Province/State"] == states[x]]
    groupC = dataC.groupby("Last Update")
    dC = groupC.sum()
    dC = list(dC["Confirmed"].values)
    
    dC.sort()
    
    dfC = pd.DataFrame({states[x]: dC})
    dfC = pd.concat([dfDate,dfC], axis=1, sort = False)
    dfC = dfC.rename(columns={0: "Date"})
    dfC = dfC.set_index('Date')
    
    df = pd.concat([df,dfC], axis=1, sort = False)
    
#df

In [ ]:
df = pd.concat([dates, df], axis=1, sort=False)

In [ ]:
#df 

In [ ]:
df[["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
    "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", 
    "ND", "OH","OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", 
    "WI", "WY", "DC"]] = df[["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN",
                             "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", 
                             "NH", "NJ", "NM", "NY", "NC", "ND", "OH","OK", "OR", "PA", "RI", "SC", "SD", "TN", 
                             "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]].interpolate()
df=df.fillna(0)
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (30, 45)

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
       "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
       "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "DC"]

fig, axs = plt.subplots(17, 3)
for i in range(17):
    axs[i,0].plot(pd.DataFrame(df[states[3*i]]))
    axs[i,0].set_title(states[3*i])
    axs[i,1].plot(pd.DataFrame(df[states[3*i+1]]))
    axs[i,1].set_title(states[3*i+1]) 
    axs[i,2].plot(pd.DataFrame(df[states[3*i+2]]))
    axs[i,2].set_title(states[3*i+2])
    
for ax in axs.flat:
    ax.set(xlabel='Date')
    ax.tick_params(labelrotation=90, axis = 'x')
       
for ax in axs.flat:
    ax.label_outer()
    
stepsize = 5
for ax in axs.flat:
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end, stepsize))
    
fig.text(0.1, 0.5, 'Confirmed Cases', ha='center', va='center', rotation='vertical', fontsize=28)

plt.text(-60, 13250, 'Boxplots for all of the States', ha='center', va='center', rotation='horizontal', fontsize=28)

In [ ]:
plt.figure(figsize=(5, 5))
state = ['NY', 'UT', 'ID', 'IL', 'WA']

dfdata = pd.DataFrame()

for x in range(len(state)):
    df1 = pd.DataFrame(df[state[x]])
    dfdata = pd.concat([dfdata, df1], axis = 1, sort = False)

dfdata    
dfdata.plot(grid = True, figsize = (10,10),title="Comparison of the Total Cases from 5 states");
plt.legend(loc=2, prop={'size': 15})

plt.text(-7, 40000, 'Confirmed Cases', ha='center', va='center', rotation='vertical', fontsize=12)
plt.text(35, -10000, 'Dates', ha='center', va='center', rotation='horizontal', fontsize=12)

Plotted a time series for the selected countries. 

# Analysis Methodology